In [49]:
import pandas as pd
import numpy as np
import scipy as sp

# Лабораторная работа №3
## Задание 2
## Кондрашин Тимофей 211-365 Вариант 11 (27.04.2023)

### Данные

Для заданного интервального выборочного ряда проверить гипотезу: закон распределения генеральной совокупности является нормальным при уровне значимости $\alpha=0.10$

**Замечание**
Начальное значение – это левая граница первого интервала, шаг – длина каждого интервала; число интервалов определяется длиной таблицы.

In [50]:
data = pd.DataFrame([2, 14, 27, 55, 74, 98, 90, 34, 14, 7])
n = len(data)
a = 0.10
min_x = 13.6
h = 1.1

data.T

,0,1,2,3,4,5,6,7,8,9
0,2,14,27,55,74,98,90,34,14,7


In [51]:
print(f'n = {n}, a = {a}, min_x = {min_x}, h = {h}')

n = 10, a = 0.1, min_x = 13.6, h = 1.1


## Шаг 1: Формулировка гипотезы

$H_0$: закон распределения генеральной совокупности является нормальным.
$H_1$: закон распределения генеральной совокупности не является нормальным.

## Шаг 2: Вычисление характеристик выборки.

### Вычисление среднего значения

$$\overline{x} = \frac{1}{n} \sum_{i=1}^{n} x_i$$

In [53]:
mean = data.mean()
mean

0    41.5
dtype: float64

### Вычисление дисперсии

$$s^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \overline{x})^2$$

In [54]:
variance = data.var()
variance

0    1256.944444
dtype: float64

### Вычисление стандартного отклонения

$$s = \sqrt{s^2}$$

In [55]:
std = data.std()
std

0    35.453412
dtype: float64

## Шаг 3: Проверка гипотезы о нормальности распределения

Для проверки гипотезы о нормальности распределения воспользуемся критерием согласия хи-квадрат. Сначала необходимо разбить интервальный ряд на интервалы.

Для этого найдём количество интервалов:

$$k = \lceil \log_2 n + 1 \rceil$$

In [56]:
k = int(np.ceil(np.log2(n) + 1))
k

5

Теперь можно разбить интервальный ряд на интервалы:

$$\Delta = \frac{max(x) - min(x)}{k}$$

In [57]:
delta = (data.max()[0] - data.min()[0]) / k
delta

19.2

Теперь можно построить интервальный ряд:

In [58]:
intervals = []
for i in range(k):
    intervals.append([min_x + i * delta, min_x + (i + 1) * delta])
intervals

[[13.6, 32.8],
 [32.8, 52.0],
 [52.0, 71.19999999999999],
 [71.19999999999999, 90.39999999999999],
 [90.39999999999999, 109.6]]

Далее, необходимо вычислить выборочные частоты попадания значений в каждый интервал:

$$p_i = \frac{n_i}{n}$$

In [59]:
p = []
for i in range(k):
    p.append(len(data[(data[0] >= min_x + i * delta) & (data[0] < min_x + (i + 1) * delta)]) / n)
p

[0.3, 0.1, 0.1, 0.2, 0.1]

Затем нужно вычислить теоретические (ожидаемые) частоты попадания значений в каждый интервал, если бы распределение было нормальным. Для этого воспользуемся формулой:
$$P_i = \Phi \left( \frac{x_{i+1} - \overline{x}}{s} \right) - \Phi \left( \frac{x_{i} - \overline{x}}{s} \right)$$

In [60]:
P = []
for i in range(k):
    P.append(sp.stats.norm.cdf((intervals[i][1] - mean) / std) - sp.stats.norm.cdf((intervals[i][0] - mean) / std))
P

[array([0.1874202]),
 array([0.21337103]),
 array([0.18245844]),
 array([0.11718923]),
 array([0.05652848])]

In [61]:
intervals_t = pd.DataFrame(intervals, columns=['min', 'max'])
p_t = pd.DataFrame(p, columns=['p (выборочные частоты)'])
P_t = pd.DataFrame(P, columns=['P (ожидаемые частоты)'])
intervals_t.join(p_t).join(P_t).T

,0,1,2,3,4
min,13.60000,32.800000,52.000000,71.200000,90.400000
max,32.80000,52.000000,71.200000,90.400000,109.600000
p (выборочные частоты),0.30000,0.100000,0.100000,0.200000,0.100000
P (ожидаемые частоты),0.18742,0.213371,0.182458,0.117189,0.056528


## Шаг 4: Вычисление значения статистики критерия

Вычислим значение статистики критерия $\chi^2$:

$$\chi^2 = \sum_{i=1}^{k} \frac{(p_i - P_i)^2}{P_i}$$

In [62]:
chi2 = ((np.array(p) - np.array(P)) ** 2 / np.array(P)).sum()
chi2

2.4610578545120574

## Шаг 5: Определение критического значения и принятие решения

#### Уровень значимости:
a = 0.1

In [65]:
a

0.1

#### Степени свободы:
$$df = k - 1$$
где
- $k$ - количество интервалов,
- $p$ - количество параметров распределения, которые были оценены по выборке.

In [66]:
df = k - 1
df

4

Таблица распределения хи-квадрат ($\chi^2$) со степенями свободы 4 и уровнем значимости 0.1 даёт критическое значение $\chi^2 = 7.779$.

Так как вычисленное значение статистики критерия Пирсона ($\chi^2 = 2.4610578545120574$) не превышает критическое значение ($\chi^2 = 7.779$), то мы принимаем нулевую гипотезу $H_0$ и отвергаем альтернативную гипотезу $H_1$. Это означает, что закон распределения генеральной совокупности является нормальным.


## Шаг 6: Вывод

Таким образом, на основе данных выборки мы можем с уверенностью утверждать, что закон распределения генеральной совокупности является нормальным.